# Segmentation

In [ ]:
import os
from modalities.test_mask_segment import  segment_brain_images, mask_brain_images
from modalities.report_segmentation import traverse_and_generate_report

def run_segmentation_and_masking(input_folder: str, tpm_path: str):
    # searching for .nii files within the subdirectories of input_folder
    subject_ids = []
    for root, _, files in os.walk(input_folder):
        for file in files:
            if file.endswith('.nii'):
                subject_ids.append(os.path.join(root, file))
    
    print(subject_ids)  # check .nii files

    for input_file in subject_ids:
        subject_id = os.path.basename(input_file).split('.')[0]
        subject_folder = os.path.dirname(input_file)
        segmentation_folder = os.path.join(subject_folder, 'segmentation')
        os.makedirs(segmentation_folder, exist_ok=True)
        
        for iteration in range(1, 4):
            segmentation_output = segment_brain_images(input_file, tpm_path, iteration, subject_id, subject_folder)
            if not segmentation_output:
                print(f"Segmentation failed for {input_file} in iteration {iteration}")
                break
            mask_output = mask_brain_images(os.path.join(segmentation_folder, f'{iteration}_segment'), subject_folder, subject_id, iteration)
            input_file = os.path.join(segmentation_folder, f'{iteration}_mask', f'masked_{subject_id}.nii')
            
            # renaming the masked output to the original file name for the next iter
            next_input_file = os.path.join(segmentation_folder, f'{iteration}_mask', f'{subject_id}.nii') 
            os.rename(input_file, next_input_file)
            input_file = next_input_file
    
    # Generate the report after all subjects have been processed
    traverse_and_generate_report(input_folder, input_folder)

# Example usage:
input_folder = 'data/analysis_conditions/'
tpm_path = 'data/TPM'
run_segmentation_and_masking(input_folder, tpm_path)

# Dartel

In [ ]:
%load_ext autoreload
%autoreload 2
import os
from modalities.dartel import create_dartel_structure, perform_dartel, copy_segment_files, find_subject_id

def main(base_path: str):
    condition_folders = [folder for folder in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, folder))]
    dartel_dir = create_dartel_structure(base_path)
    copy_segment_files(base_path, condition_folders, dartel_dir)
    perform_dartel(dartel_dir)

main('/home/sharapova/vbm_pipline/data/analysis_conditions')

# Normalization

In [ ]:
%load_ext autoreload
%autoreload 2
import os
from modalities.normalization import normalize_brain_images

root_derivatives_dir_path = os.path.abspath('data/analysis_/control_analysis/')
output_files = normalize_brain_images(root_derivatives_dir_path)
print(output_files)

# Smoothing

In [ ]:
%load_ext autoreload
%autoreload 2
import os
from modalities.smoothing import smooth_brain_images

root_derivatives_dir_path = os.path.abspath('/home/sharapova/vbm_pipline/data/analysis_conditions')

# Define the full width at half maximum (FWHM) for smoothing
fwhm = [3, 3, 3] 


output_files = smooth_brain_images(root_derivatives_dir_path, fwhm)
print(output_files)

# Statistical modeling and estimation

In [ ]:
%load_ext autoreload
%autoreload 2
import os
from modalities.statistical_modeling_t_test_model import create_two_sample_t_test_model

contrasts_name = 'example_contrast_new_logic'
output_files = create_two_sample_t_test_model('/home/sharapova/vbm_pipline/data/analysis_conditions', contrasts_name)
print(output_files)